In [1]:
#Import things
import astropy
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import astropy.units as u
import numpy as np

In [5]:
# Gaia Query

coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
r = Gaia.query_object_async(coordinate=coord, width=width, height=height)

# r.pprint(max_lines=12, max_width=130)
# print(type(r))
r.info

INFO: Query finished. [astroquery.utils.tap.core]


<Table length=50>
              name                dtype       unit                                          description                                      n_bad
-------------------------------- ------- ------------- ------------------------------------------------------------------------------------- -----
                            dist float64                                                                                                         0
                     solution_id   int64                                                                                 Solution Identifier     0
                     DESIGNATION  object                                         Unique source designation (unique across all Data Releases)     0
                       source_id   int64                                  Unique source identifier (unique within a particular Data Release)     0
                    random_index   int64                                                            

In [28]:
# ATNF Query
from psrqpy import QueryATNF
query = QueryATNF(condition= 'PMRA > 0')
table = query.table
print(table)

    PSRJ    PSRJ_REF        RAJ        ... TYPE_ORIG  TYPE_REF  BINCOMP_ORIG
                                       ...                                  
----------- -------- ----------------- ... ---------- -------- -------------
 J0014+4746       --       00:14:17.75 ...         --                     --
J0024-7204C       --     00:23:50.3546 ...         --                     --
J0024-7204D       --    00:24:13.88092 ...         --                     --
J0024-7204E       --    00:24:11.10528 ...         --                     --
J0024-7204F       --    00:24:03.85547 ...         --                     --
J0024-7204G       --     00:24:07.9603 ...         --                     --
J0024-7204H       --     00:24:06.7032 ...         --             He[fck+03]
J0024-7204I       --     00:24:07.9347 ...         --             UL[fck+03]
J0024-7204J       --     00:23:59.4077 ...         --             UL[fck+03]
J0024-7204L       --     00:24:03.7721 ...         --                     --

In [29]:
print(table.info)

<Table length=201>
    name      dtype       unit         class     n_bad
------------ ------- -------------- ------------ -----
        PSRJ   str12                      Column     0
    PSRJ_REF  object                MaskedColumn   106
         RAJ   str17                      Column     0
     RAJ_ERR float64                MaskedColumn     1
     RAJ_REF    str7                      Column     0
        DECJ   str18                      Column     0
    DECJ_ERR float64                      Column     0
    DECJ_REF    str7                      Column     0
    POSEPOCH float64              d       Column     0
POSEPOCH_REF  object                MaskedColumn     3
          F0 float64             Hz       Column     0
      F0_ERR float64             Hz MaskedColumn     1
      F0_REF    str7                      Column     0
          F1 float64         1 / s2 MaskedColumn     1
      F1_ERR float64         1 / s2 MaskedColumn     1
      F1_REF    str7                      Colu

Given an item from ATNF catalog, find nearby Gaia objects

In [31]:
from astropy.time import Time

# Gather information on a given pulsar

pulsar = 1 # check first pulsar listed in ATNF

p_ascension = table["RAJ"][pulsar]
p_declination = table["DECJ"][pulsar]
p_pmra = table["PMRA"][pulsar]
p_pmdec = table["PMDEC"][pulsar]

print (p_ascension + " " + p_declination)

raw_epoch = table["POSEPOCH"][pulsar]
p_epoch = Time(raw_epoch, format='mjd').jyear

print(p_epoch) # testing that p_epoch is now in the correct time units, jyear


00:23:50.3546 -72:04:31.5048
2000.1519507186858
<class 'numpy.str_'>
<class 'float'>


In [55]:
p_ascension_angle = Angle(p_ascension.tolist(), u.degree) # convert p_ascension to Angle object 
p_declination_angle = Angle(p_declination.tolist(), u.degree) # convert p_declination to Angle object

print(type(p_declination))
print(type(p_epoch.tolist()))


<class 'numpy.str_'>
<class 'float'>


In [42]:
gaia_epoch = 2015.5 * u.yr # define gaia epoch in units years
year_diff = gaia_epoch - p_epoch.tolist() * u.yr # define the time difference in years as a float

p_pmra_deg = (p_pmra.tolist() * u.mas).to(u.deg) / u.yr # convert p_pmra from mas/yr to deg/yr
p_pmdec_deg = (p_pmdec.tolist() * u.mas).to(u.deg) / u.yr # convert p_pmdec_deg from mas/yr to deg/yr

assert p_pmra_deg.unit.is_equivalent(u.deg / u.yr)
assert p_pmdec_deg.unit.is_equivalent(u.deg / u.yr)
assert year_diff.unit.is_equivalent(u.yr)

In [51]:
# Update location of pulsar based on epoch difference with Gaia

gaia_epoch = 2015.5 * u.yr # define gaia epoch in units years
year_diff = gaia_epoch - p_epoch.tolist() * u.yr # define the time difference in years as a float

In [54]:
p_new_ascension = p_ascension_angle + (p_pmra_deg * year_diff)
p_new_declination = p_declination_angle + (p_pmdec_deg * year_diff)

print(p_ascension)
print(p_new_ascension)
print(p_declination)
print(p_new_declination)


00:23:50.3546
0d23m50.43440986s
-72:04:31.5048
-72d04m31.55237895s


In [57]:
print(table['JNAME'][pulsar])

J0024-7204C


In [56]:
# Querying Gaia

coord = SkyCoord(ra=p_new_ascension, dec=p_new_declination, unit=(u.degree, u.degree), frame='icrs')
width = u.Quantity(0.1, u.deg)
height = u.Quantity(0.1, u.deg)
results = Gaia.query_object_async(coordinate=coord, width=width, height=height)
results.show_in_notebook()

INFO: Query finished. [astroquery.utils.tap.core]


idx,dist,solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url
,,,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,
0,0.010770322091869818,1635721458409799680,Gaia DR2 4701777458586819200,4701777458586819200,1326602477,2015.5,0.3670381781132863,0.1750663049955098,-72.07003903935032,0.14415774246632043,0.8528259584063638,0.17363713915886217,4.911541,16.293505517485777,0.3557492506397365,2.2520780613341005,0.27396779219432926,-0.37166435,-0.037863515,-0.29874796,0.27259913,0.0037610917,0.2881091,-0.21209538,-0.04948961,-0.29824668,-0.3498872,243,0,242,1,0.7209974,252.35054,0.0,0.0,31,False,0.41161847,1.4592049190896463,0.04426088762427587,-0.036683906,28,19,0.34636697,0,31,False,265,640.4724934715656,1.2989646151958298,493.06384,18.672115,27,206.7217962784954,6.842488921576501,30.21149,19.562923,26,680.0966812208154,7.848070450449832,86.65782,17.680492,1.3846316,0,1.882431,0.8908081,0.9916229,--,--,0,--,--,--,NOT_AVAILABLE,308.2922857138715,-44.53445577271068,309.2602870446957,-60.89240842033204,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+4701777458586819200
1,0.012565233225270905,1635721458409799680,Gaia DR2 4701777458586818688,4701777458586818688,1516198541,2015.5,0.3579056019514615,0.590241224214455,-72.07216941089214,0.5210866749517815,0.18140390046207316,0.5979577913063845,0.3033724,-0.9108519307338189,1.26165376465947,-2.22980585705494,1.1178086684556152,-0.39034683,-0.24355683,-0.3624378,0.33152732,0.121275984,0.28114042,-0.22509238,-0.023792414,-0.22842841,-0.41262785,218,0,217,1,0.29940933,217.54622,0.0,0.0,31,False,0.039029423,1.454741995342903,0.1551772287899112,-0.019495798,25,18,1.2912309,0,28,False,242,136.34472825248628,1.0486448846502086,130.01993,20.35177,24,102.4722610376602,7.545858911322591,13.579933,20.324873,21,86.03343709257243,8.406689054028883,10.233927,19.925251,1.3825668,0,0.39962196,-0.02689743,0.4265194,--,--,0,--,--,--,NOT_AVAILABLE,308.2952755001154,-44.531645103868485,309.25337922937814,-60.89133602944594,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,https://gea.esac.esa.int/da